<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session3_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 3 (Morphology) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [14]:
%%shell
git clone https://github.com/mariorot/IHLT-MAI.git
cd 'IHLT-MAI'
mv 'complementary_material' /content/
mv scripts /content/

fatal: destination path 'IHLT-MAI' already exists and is not an empty directory.
mv: cannot stat 'complementary_material': No such file or directory
mv: cannot stat 'scripts': No such file or directory


CalledProcessError: ignored

In [25]:
from scripts.compute_metrics import ComputeMetrics
from scripts.text_preprocessing import TextPreprocessing
import pandas as pd
pd.set_option("display.precision", 4)
from scipy.stats import pearsonr

In [16]:
dt = pd.read_csv('complementary_material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
dt['gs'] = pd.read_csv('complementary_material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

## TODO
1. Read all pairs of sentences of the SMTeuroparl files of test set within the evaluation framework of the project. Compute their similarities by considering *lemmas* and *Jaccard distance*. <br>

2. Compare the results with those in session 2 (document structure) in which words were considered. : <br>

3. Compare the results with gold standard by giving the pearson correlation between them. <br>

4. Questions (justify the answers): <br>

      Which is better: words or lemmas?

      Do you think that could perform better for any pair of texts?

In [17]:
tp = TextPreprocessing()

# -- Data cleaning
dt[2]=tp.clean_data(list(dt[0]))
dt[3]=tp.clean_data(list(dt[1]))

# -- Tokenization
# NLTK
dt[4] = tp.tokenize_data(list(dt[0]),'nltk')
dt[5] = tp.tokenize_data(list(dt[1]),'nltk')
# spaCy
dt[6] = tp.tokenize_data(list(dt[0]),'spacy')
dt[7] = tp.tokenize_data(list(dt[1]),'spacy')

# -- Lemmatization
# NLTK
dt[8]=tp.lemmatize_data(list(dt[0]),'nltk',False)
dt[9]=tp.lemmatize_data(list(dt[1]),'nltk',False)
# spaCy
dt[10]=tp.lemmatize_data(list(dt[0]),'spacy')
dt[11]=tp.lemmatize_data(list(dt[1]),'spacy')


# -- Lemmatization with cleaned data
# NLTK
dt[12]=tp.lemmatize_data(list(dt[2]),'nltk')
dt[13]=tp.lemmatize_data(list(dt[3]),'nltk')
# spaCy
dt[14]=tp.lemmatize_data(list(dt[2]),'spacy')
dt[15]=tp.lemmatize_data(list(dt[3]),'spacy')

The code for the tokenization is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/text_preprocessing.py

The code for jaccard distance computation is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/compute_metrics.py

In [18]:
# -- Jaccard computation
# Tokens
cm_clean_nltk = ComputeMetrics(dt[[2,3]].to_numpy(), ['jaccard'], 1)
dt['jaccard_clean'] = cm_clean_nltk.do()['jaccard']

cm_nltk = ComputeMetrics(dt[[4,5]].to_numpy(), ['jaccard'], 1)
dt['jaccard_token_nltk'] = cm_nltk.do()['jaccard']

cm_spacy = ComputeMetrics(dt[[6,7]].to_numpy(), ['jaccard'], 1)
dt['jaccard_token_spacy'] = cm_spacy.do()['jaccard']

# Lemmas
cm_lemma_nltk = ComputeMetrics(dt[[8,9]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_nltk']=cm_lemma_nltk.do()['jaccard']

cm_lemma_spacy = ComputeMetrics(dt[[10,11]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_spacy']=cm_lemma_spacy.do()['jaccard']

cm_lemma_clean_nltk = ComputeMetrics(dt[[12,13]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_clean_nltk']=cm_lemma_clean_nltk.do()['jaccard']

cm_lemma_clean_spacy = ComputeMetrics(dt[[14,15]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_clean_spacy']=cm_lemma_clean_spacy.do()['jaccard']

Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...


In [27]:
dt[['gs','jaccard_token_nltk','jaccard_token_spacy','jaccard_clean','jaccard_lemma_nltk','jaccard_lemma_clean_nltk','jaccard_lemma_spacy','jaccard_lemma_clean_spacy']]

,gs,jaccard_token_nltk,jaccard_token_spacy,jaccard_clean,jaccard_lemma_nltk,jaccard_lemma_clean_nltk,jaccard_lemma_spacy,jaccard_lemma_clean_spacy
0,4.500,0.6538,0.6538,0.6923,0.6538,0.6667,0.6000,0.6667
1,5.000,0.2143,0.2143,0.2857,0.0769,0.0000,0.0769,0.0000
2,4.250,0.6087,0.6087,0.7273,0.6087,0.7273,0.5455,0.7273
3,4.500,0.4545,0.4000,0.2500,0.4545,0.2500,0.4000,0.2500
4,5.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...
454,5.000,0.4500,0.4500,0.6364,0.4500,0.6364,0.4500,0.6364
455,4.750,0.6429,0.7143,0.2500,0.6429,0.2500,0.6154,0.2500
456,5.000,0.6000,0.6000,0.6667,0.6000,0.6667,0.5263,0.6667
457,4.000,0.6818,0.6818,0.8000,0.6818,0.8000,0.6190,0.8000


In [26]:
results = {'Category':['NLTK', 'spaCy', 'NLTK Clean', 'spaCy Clean'],'Token':[], 'Lemma':[]}
results['Token'].append(pearsonr(dt['gs'], 1-dt['jaccard_token_nltk'])[0])
results['Token'].append(pearsonr(dt['gs'], 1-dt['jaccard_token_spacy'])[0])
results['Token'].append(pearsonr(dt['gs'], 1-dt['jaccard_clean'])[0])
results['Token'].append(pearsonr(dt['gs'], 1-dt['jaccard_clean'])[0])
results['Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_nltk'])[0])
results['Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_clean_nltk'])[0])
results['Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_spacy'])[0])
results['Lemma'].append(pearsonr(dt['gs'], 1-dt['jaccard_lemma_clean_spacy'])[0])

results = pd.DataFrame(results)
results.set_index('Category')

,Token,Lemma
Category,,
NLTK,0.4505,0.4493
spaCy,0.4609,0.4810
NLTK Clean,0.4679,0.4768
spaCy Clean,0.4679,0.4912


# Conclusion

In this lab session excercise, we compared sentence pairs using Jaccard distance with and without lemmatization and data cleaning.

- **Which is better: words or lemmas?**
  - It can be seen that on average the lemmas outperformed the words in 3 out of 4 scenarios by 2-5%.
  
 -  The only case in which the words were better was with the raw/uncleaned pair of sentences with the NLTK tokenizer, but only by 0.2%.
  
  - Thus, in general, lemmas work better with either NLTK or spaCy lemmatization, and this makes sense because we have the root of the words, and the similarity will be higher, instead of possible variations where the similarity will be lower.


- **Do you think that could perform better for any pair of texts?**

  - Even when we are going to have exceptions with some sentences, in the majority of the cases the performance will be better because we increase the number of words in common between the compared sets.


SpaCy's lemmatization generally outperformed NLTK, indicating that it might be more effective in capturing the semantic similarity between words, leading to improved sentence comparison results.

Data cleaning including lowercase conversion, punctuation removal, and stopword removal, helped in all cases to improve performance. Both NLTK and spaCy lemmatization processes benefited from data cleaning, resulting in several cases in lower Jaccard distance values, indicating a better measure of similarity.

